In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('../input/tabular-playground-series-oct-2021/train.csv')

In [ ]:
import sklearn

In [ ]:
from sklearn import datasets
from sklearn import model_selection

def create_folds(data):
 # we create a new column called kfold and fill it with -1
 data["kfold"] = -1
 
 # the next step is to randomize the rows of the data
 data = data.sample(frac=1).reset_index(drop=True)
 # calculate the number of bins by Sturge's rule
 # I take the floor of the value, you can also
 # just round it
#  num_bins = int(np.floor(1 + np.log2(len(data))))
#  # bin targets
#  data.loc[:, "bins"] = pd.cut(
#  data["target"], bins=num_bins, labels=False
#  )
 
 # initiate the kfold class from model_selection module
 kf = model_selection.StratifiedKFold(n_splits=5)
 
 # fill the new kfold column
 # note that, instead of targets, we use bins!
 for f, (t_, v_) in enumerate(kf.split(X=data, y=data.target)):
     data.loc[v_, 'kfold'] = f
 
 # drop the bins column
#  data = data.drop("bins", axis=1)
 # return dataframe with folds
 return data

In [ ]:
df=create_folds(df)

In [ ]:
import optuna
from sklearn.ensemble import RandomForestClassifier
import xgboost as xg
import joblib
import lightgbm as lgb 

In [ ]:
def objective(trial):
    
    param = {
        'tree_method':'gpu_hist',  # this parameter means using the GPU when training our model to speedup the training process
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.009,0.01,0.012,0.014,0.016,0.018, 0.02]),
        'n_estimators': 4000,
        'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17,20]),
        'random_state': trial.suggest_categorical('random_state', [24, 48,2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
        'use_label_encoder':False
    }
    
    
    roc_auc_score=0
    clf = xg.XGBClassifier(**param)
    for fold in range(5):
        
        df_train=df[df.kfold!=fold]
        df_test=df[df.kfold==fold]

        x_train=df_train.drop(['target','id','kfold'],axis=1).values
        y_train=df_train.target.values

        x_test=df_test.drop(['target','id','kfold'],axis=1).values
        y_test=df_test.target.values

    #     clf=RandomForestClassifier()
        clf.fit(x_train,y_train)

        y_pred = clf.predict_proba(x_test)[:,1]
        roc_auc_score = roc_auc_score+sklearn.metrics.roc_auc_score(y_test,y_pred)
    
    return roc_auc_score/5
#     return sklearn.model_selection.cross_val_score(clf, x, y, 
#        n_jobs=-1, cv=5).mean()

In [ ]:
study = optuna.create_study(direction='maximize')
# optimization_function=partial(objective,x=x,y=y)
study.optimize(objective, n_trials=12)

i didnot run this here because i have done it already in my local 

In [ ]:
def run_folds(df,fold):
    df_train=df[df.kfold!=fold]
    df_test=df[df.kfold==fold]
    
    x_train=df_train.drop(['target','id','kfold'],axis=1).values
    y_train=df_train.target.values
    
    x_test=df_test.drop(['target','id','kfold'],axis=1).values
    y_test=df_test.target.values
    #xgboost
    param={'tree_method':'gpu_hist','lambda': 0.5033009770384954, 'alpha': 0.5585785710700676, 'colsample_bytree': 0.8, 
           'subsample': 0.5, 'learning_rate': 0.008, 'max_depth': 11, 'random_state': 2020, 'min_child_weight': 263,
          'use_label_encoder':False,'n_estimators': 4000}

    
    clf= xg.XGBClassifier(**param)
    clf.fit(x_train,y_train)
    
    y_pred = clf.predict_proba(x_test)[:,1]
    roc_auc_score = sklearn.metrics.roc_auc_score(y_test,y_pred)
    print(f"Fold={fold}, roc_auc_score={roc_auc_score}")
    
    File_name = 'model_lgbmclf' + str(fold)
    joblib.dump(
    clf,File_name)
    
    
for i in range(5):
    run_folds(df,i)    

In [ ]:
# i didnot run the aboe code cell also just to save time

In [ ]:
param={'tree_method':'gpu_hist','lambda': 0.5033009770384954, 'alpha': 0.5585785710700676, 'colsample_bytree': 0.8, 
           'subsample': 0.5, 'learning_rate': 0.008, 'max_depth': 11, 'random_state': 2020, 'min_child_weight': 263,
          'use_label_encoder':False,'n_estimators': 4000}
xgclf=xg.XGBClassifier(**param)
xgclf.fit(df.iloc[:,1:-2],df.target)

In [ ]:
submission = pd.read_csv('../input/tabular-playground-series-oct-2021/sample_submission.csv')

In [ ]:
df_test = pd.read_csv('../input/tabular-playground-series-oct-2021/test.csv')
df_test = df_test.drop(columns = 'id')

In [ ]:
y_final_avg=xgclf.predict_proba(df_test)

In [ ]:
submission['target'] = y_final_avg

In [ ]:
submission.to_csv('pred_csv_xg_optuna_tuned2.csv',index = False)